# Analysis of DNA-MERFISH for CTP11

by Cosmos Wang

2023.01.27

analysis for dataset:

\\10.245.74.158\Chromatin_NAS_8\20230119_Ma_1027_DNA
\\10.245.74.158\Chromatin_NAS_6\20230119_Ma_1027_DNA

This data is DNA of uncleared MERFISH RNA:
    \\10.245.74.158\Chromatin_NAS_6\20221027_Ma_withDMG

In [1]:
%run "..\..\Startup_py3.py"
sys.path.append(r"..\..\..\..\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

27244


# 0. fov parameters

In [2]:
reload(ia)
reload(classes)
reload(classes.batch_functions)
reload(classes.field_of_view)
reload(io_tools.load)
reload(get_img_info)
reload(visual_tools)
reload(ia.correction_tools)
reload(ia.correction_tools.alignment)
reload(ia.spot_tools.matching)
reload(ia.segmentation_tools.chromosome)
reload(ia.spot_tools.fitting)

<module 'ImageAnalysis3.spot_tools.fitting' from 'E:\\Users\\puzheng_new\\Documents\\Chromatin_Analysis_Scripts\\Tissue_DNA-FISH\\MECP2\\..\\..\\..\\..\\Documents\\ImageAnalysis3\\spot_tools\\fitting.py'>

In [3]:
fov_param = {'data_folder':[r'\\10.245.74.158\Chromatin_NAS_8\20230119_Ma_1027_DNA',
                            r'\\10.245.74.158\Chromatin_NAS_6\20230119_Ma_1027_DNA',
                            r'\\10.245.74.158\Chromatin_NAS_7\20230119_Ma_1027_DNA',
                            r'\\10.245.74.158\Chromatin_NAS_5\20230119_Ma_1027_DNA',
                            r'\\10.245.74.158\Chromatin_NAS_1\20230119_Ma_1027_DNA',
                           ],
             'save_folder':r'F:\DNA_analysis\20230119_Ma_1027_DNA',
             'experiment_type': 'DNA',
             'num_threads': 44,
             'correction_folder':r'\\10.245.74.158\Chromatin_NAS_0\Corrections\20210621-Corrections_lumencor_from_60_to_50',
             'shared_parameters':{
                     'single_im_size':[50,2048,2048],
                     'distance_zxy': [250, 108, 108],
                     'corr_channels':['750','647','561'],
                     'num_empty_frames': 0, 
                     'num_buffer_frames':0,
                     'corr_hot_pixel':True,
                     'corr_Z_shift':False,
                     'corr_bleed':True,
                     'min_num_seeds':200,
                     'max_num_seeds': 20000,
                     'spot_seeding_th': 800,
                     'normalize_intensity_local':False,
                     'normalize_intensity_background':False,
                     'corr_gaussian_highpass':False,
                 }, 
             }

# 1. Process Fov

In [ ]:
_overwrite = False

_save_images = False

_fit_spots = True

bad_fov_ids = [0, 221]

sel_fov_ids = np.arange(100,222) # Pu Zheng in Mendel
#sel_fov_ids = np.arange(0,100) # cosmos

for _fov_id in sel_fov_ids:
    if 'bad_fov_ids' in locals() and  _fov_id in bad_fov_ids:
        continue
        
    if _fov_id in np.arange(100,102):
        _save_images = True
    else:
        _save_images = False
    
    fov = classes.field_of_view.Field_of_View(fov_param, _fov_id=_fov_id,
                                              _color_info_kwargs={
                                                  '_color_filename':'Color_Usage',
                                              }, 
                                              _prioritize_saved_attrs=False,
                                              _save_info_to_file=False, # whether overwrite
                                              )
    
    with h5py.File(fov.save_filename, 'a') as _f:
        if 'relabeled_combo' in _f.keys() and len(_f['relabeled_combo']['ids'][:]) <=20:
            print(f"delete old in {_fov_id}")
            del _f['relabeled_combo']
            # recreate
        fov._save_to_file('relabeled_combo')
        
    # 2. Process image into candidate spots
    fov.parallel = True
    setattr(fov, "combo_ref_id", 0)
    fov._process_image_to_spots('combo', 
                                _load_common_reference=True, _load_with_multiple=False,
                                _save_images=_save_images,
                                _warp_images=False, _fit_spots=True,
                                _overwrite_drift=False, _overwrite_image=_overwrite,
                                _overwrite_spot=_overwrite)
    
    # 3. relabeled_combo
    fov.parallel = True
    setattr(fov, "relabeled_combo_ref_id", 0)
    fov._process_image_to_spots('relabeled_combo', 
                                _load_common_reference=True, _load_with_multiple=False,
                                _save_images=_save_images,
                                _warp_images=False, _fit_spots=True,
                                _overwrite_drift=False, _overwrite_image=_overwrite,
                                _overwrite_spot=_overwrite)
    
    # 4. Process DAPI image
    fov._load_dapi_image()

Get Folder Names: (ia.get_img_info.get_folders)
- Number of folders: 46
- Number of field of views: 222
Get Folder Names: (ia.get_img_info.get_folders)
- Number of folders: 27
- Number of field of views: 222
Get Folder Names: (ia.get_img_info.get_folders)
- Number of folders: 12
- Number of field of views: 222
Get Folder Names: (ia.get_img_info.get_folders)
- Number of folders: 25
- Number of field of views: 222
Get Folder Names: (ia.get_img_info.get_folders)
- Number of folders: 21
- Number of field of views: 222
- Importing csv format color_usage file: \\10.245.74.158\Chromatin_NAS_8\20230119_Ma_1027_DNA\Analysis\Color_Usage.csv
- header: ['Hyb', '750', '647', '561', '488', '405']
- 131 folders are found according to color-usage annotation.
+ loading fov_info from file: F:\DNA_analysis\20230119_Ma_1027_DNA\Conv_zscan_100.hdf5
++ base attributes loaded:['combo_ref_im', 'dapi_im', 'relabeled_combo_ref_im'] in 5.374s.
+ loading correction from file: F:\DNA_analysis\20230119_Ma_1027_DNA\

-- checking relabeled_combo, region:[285 286 287] in 0.048s.
-- checking relabeled_combo, region:[381 382 383] in 0.053s.
-- checking relabeled_combo, region:[288 289 290] in 0.049s.
-- checking relabeled_combo, region:[384 385 386] in 0.055s.
-- checking relabeled_combo, region:[291 292 293] in 0.048s.
-- checking relabeled_combo, region:[387 388 389] in 0.053s.
-- checking relabeled_combo, region:[294 295 296] in 0.048s.
-- checking relabeled_combo, region:[390 391 392] in 0.051s.
- No relabeled_combo images and spots requires processing, skip.
directly return existing attribute.
Get Folder Names: (ia.get_img_info.get_folders)
- Number of folders: 46
- Number of field of views: 222
Get Folder Names: (ia.get_img_info.get_folders)
- Number of folders: 27
- Number of field of views: 222
Get Folder Names: (ia.get_img_info.get_folders)
- Number of folders: 12
- Number of field of views: 222
Get Folder Names: (ia.get_img_info.get_folders)
- Number of folders: 25
- Number of field of views:

-- checking relabeled_combo, region:[360 361 362] in 0.024s.
-- checking relabeled_combo, region:[267 268 269] in 0.027s.
-- checking relabeled_combo, region:[363 364 365] in 0.025s.
-- checking relabeled_combo, region:[270 271 272] in 0.025s.
-- checking relabeled_combo, region:[366 367 368] in 0.024s.
-- checking relabeled_combo, region:[273 274 275] in 0.022s.
-- checking relabeled_combo, region:[369 370 371] in 0.022s.
-- checking relabeled_combo, region:[276 277 278] in 0.022s.
-- checking relabeled_combo, region:[372 373 374] in 0.021s.
-- checking relabeled_combo, region:[279 280 281] in 0.024s.
-- checking relabeled_combo, region:[375 376 377] in 0.024s.
-- checking relabeled_combo, region:[282 283 284] in 0.023s.
-- checking relabeled_combo, region:[378 379 380] in 0.022s.
-- checking relabeled_combo, region:[285 286 287] in 0.021s.
-- checking relabeled_combo, region:[381 382 383] in 0.022s.
-- checking relabeled_combo, region:[288 289 290] in 0.021s.
-- checking relabeled_co

-- checking relabeled_combo, region:[279 280 281] in 0.008s.
-- checking relabeled_combo, region:[375 376 377] in 0.007s.
-- checking relabeled_combo, region:[282 283 284] in 0.007s.
-- checking relabeled_combo, region:[378 379 380] in 0.008s.
-- checking relabeled_combo, region:[285 286 287] in 0.007s.
-- checking relabeled_combo, region:[381 382 383] in 0.008s.
-- checking relabeled_combo, region:[288 289 290] in 0.007s.
-- checking relabeled_combo, region:[384 385 386] in 0.008s.
-- checking relabeled_combo, region:[291 292 293] in 0.007s.
-- checking relabeled_combo, region:[387 388 389] in 0.008s.
-- checking relabeled_combo, region:[294 295 296] in 0.007s.
-- checking relabeled_combo, region:[390 391 392] in 0.008s.
+ Start multi-processing of pre-processing for 64 images with 44 threads
++ processing relabeled_combo ids: [201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218
 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236
 237 238 239 2

-- checking relabeled_combo, region:[252 253 254] in 0.008s.
-- checking relabeled_combo, region:[348 349 350] in 0.008s.
-- checking relabeled_combo, region:[255 256 257] in 0.007s.
-- checking relabeled_combo, region:[353 352 351] in 0.008s.
-- checking relabeled_combo, region:[258 259 260] in 0.007s.
-- checking relabeled_combo, region:[354 355 356] in 0.008s.
-- checking relabeled_combo, region:[261 262 263] in 0.007s.
-- checking relabeled_combo, region:[357 358 359] in 0.008s.
-- checking relabeled_combo, region:[264 265 266] in 0.007s.
-- checking relabeled_combo, region:[360 361 362] in 0.008s.
-- checking relabeled_combo, region:[267 268 269] in 0.007s.
-- checking relabeled_combo, region:[363 364 365] in 0.008s.
-- checking relabeled_combo, region:[270 271 272] in 0.008s.
-- checking relabeled_combo, region:[366 367 368] in 0.007s.
-- checking relabeled_combo, region:[273 274 275] in 0.008s.
-- checking relabeled_combo, region:[369 370 371] in 0.007s.
-- checking relabeled_co

...

-- checking combo, region:[57 58 95] in 0.049s.
-- checking combo, region:[59 60 96] in 0.049s.
-- checking combo, region:[61 62 97] in 0.048s.
-- checking combo, region:[63 64 98] in 0.049s.
-- checking combo, region:[65 66 99] in 0.051s.
-- checking combo, region:[151 152 190] in 0.081s.
-- checking combo, region:[153 154 191] in 0.057s.
-- checking combo, region:[155 156 192] in 0.057s.
-- checking combo, region:[157 158 193] in 0.069s.
-- checking combo, region:[159 160 194] in 0.055s.
-- checking combo, region:[161 162 195] in 0.056s.
-- checking combo, region:[163 164] in 0.039s.
-- checking combo, region:[175 168] in 0.049s.
-- checking combo, region:[179 176] in 0.037s.
-- checking combo, region:[101 102 165] in 0.057s.
-- checking combo, region:[103 104 166] in 0.054s.
-- checking combo, region:[105 106 167] in 0.069s.
-- checking combo, region:[107 108] in 0.041s.
-- checking combo, region:[109 110 169] in 0.057s.
-- checking combo, region:[111 112 170] in 0.057s.
-- checking

- 131 folders are found according to color-usage annotation.
+ loading fov_info from file: F:\DNA_analysis\20230119_Ma_1027_DNA\Conv_zscan_134.hdf5
++ base attributes loaded:['combo_ref_im', 'dapi_im'] in 5.892s.
+ loading correction from file: F:\DNA_analysis\20230119_Ma_1027_DNA\Conv_zscan_134.hdf5
++ load bleed correction profile directly from savefile.
++ load chromatic correction profile directly from savefile.
++ load chromatic_constants correction profile directly from savefile.
++ load illumination correction profile directly from savefile.
+ loading segmentation from file: F:\DNA_analysis\20230119_Ma_1027_DNA\Conv_zscan_134.hdf5
++ base attributes loaded:[] in 0.007s.
-- saving relabeled_combo to file: F:\DNA_analysis\20230119_Ma_1027_DNA\Conv_zscan_134.hdf5
--- relabeled_combo attributes updated:['ids', 'channels', 'ims', 'spots', 'raw_spots', 'drifts', 'flags'] in 0.011s.
-- folders not selected, allow processing all 131 folders
-- checking combo, region:[ 1  2 67] in 0.060s

-- checking relabeled_combo, region:[387 388 389] in 0.008s.
-- checking relabeled_combo, region:[294 295 296] in 0.008s.
-- checking relabeled_combo, region:[390 391 392] in 0.007s.
+ Start multi-processing of pre-processing for 64 images with 44 threads
++ processing relabeled_combo ids: [201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218
 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236
 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254
 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272
 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290
 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308
 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326
 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344
 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362
 363 364 365 366 367 368 369 370 371 372 373 374 375

-- checking relabeled_combo, region:[252 253 254] in 0.008s.
-- checking relabeled_combo, region:[348 349 350] in 0.008s.
-- checking relabeled_combo, region:[255 256 257] in 0.007s.
-- checking relabeled_combo, region:[353 352 351] in 0.007s.
-- checking relabeled_combo, region:[258 259 260] in 0.007s.
-- checking relabeled_combo, region:[354 355 356] in 0.008s.
-- checking relabeled_combo, region:[261 262 263] in 0.007s.
-- checking relabeled_combo, region:[357 358 359] in 0.008s.
-- checking relabeled_combo, region:[264 265 266] in 0.007s.
-- checking relabeled_combo, region:[360 361 362] in 0.008s.
-- checking relabeled_combo, region:[267 268 269] in 0.008s.
-- checking relabeled_combo, region:[363 364 365] in 0.007s.
-- checking relabeled_combo, region:[270 271 272] in 0.008s.
-- checking relabeled_combo, region:[366 367 368] in 0.008s.
-- checking relabeled_combo, region:[273 274 275] in 0.007s.
-- checking relabeled_combo, region:[369 370 371] in 0.007s.
-- checking relabeled_co

-- loading image from file:\\10.245.74.158\Chromatin_NAS_8\20230119_Ma_1027_DNA\H0C1\Conv_zscan_136.dax  in 9.747s
-- removing hot pixels for channels:['488'] in 9.220s
-- illumination correction for channels: 488, in 2.916s
-- -- generate translation function with drift:[0. 0. 0.] in 0.000s
-- finish correction in 22.816s
-- saving fov_info to file: F:\DNA_analysis\20230119_Ma_1027_DNA\Conv_zscan_136.hdf5
++ base attributes saved:['relabeled_combo_ref_im'] in 4.452s.
-- checking relabeled_combo, region:[201 202 203] in 0.013s.
-- checking relabeled_combo, region:[297 298 299] in 0.008s.
-- checking relabeled_combo, region:[204 205 206] in 0.008s.
-- checking relabeled_combo, region:[300 301 302] in 0.008s.
-- checking relabeled_combo, region:[207 208 209] in 0.008s.
-- checking relabeled_combo, region:[303 304 305] in 0.008s.
-- checking relabeled_combo, region:[210 211 212] in 0.008s.
-- checking relabeled_combo, region:[306 307 308] in 0.008s.
-- checking relabeled_combo, region:[21

In [26]:
fov._load_from_file('relabeled_combo')

+ loading relabeled_combo from file: F:\DNA_analysis\20230119_Ma_1027_DNA\Conv_zscan_100.hdf5
++ finish loading relabeled_combo in 0.469s. 


In [28]:
len(fov.relabeled_combo_ids)

192

-- saving relabeled_combo to file: F:\DNA_analysis\20230119_Ma_1027_DNA\Conv_zscan_100.hdf5
--- relabeled_combo attributes updated:['ids', 'channels', 'ims', 'spots', 'raw_spots', 'drifts', 'flags', 'relabeled_combo_ref_im'] in 1.179s.
